# Exploration of Dataset

In [1]:
import pandas as pd
import numpy as np

# Load the datasets
train_df = pd.read_csv('./datasets/train.csv')
valid_df = pd.read_csv('./datasets/valid.csv')
test_df = pd.read_csv('./datasets/test.csv')

# Display dataset shapes
print(f"Train Shape: {train_df.shape}")
print(f"Valid Shape: {valid_df.shape}")
print(f"Test Shape: {test_df.shape}")
print()

# Preview the training data
print(train_df.head())
print()

# Check for class balance in the training set
print(train_df['label'].value_counts())
print()

# Check for any missing values
print(train_df.isnull().sum())


Train Shape: (21464, 2)
Valid Shape: (716, 2)
Test Shape: (966, 2)

                                                text  label
0  states slow to shut down weak teacher educatio...      0
1    drone places fresh kill on steps of white house      1
2  report: majority of instances of people gettin...      1
3  sole remaining lung filled with rich, satisfyi...      1
4                       the gop's stockholm syndrome      0

label
0    11248
1    10216
Name: count, dtype: int64

text     0
label    0
dtype: int64


/Users/dylanwong/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Baseline Model with Bag of Words and Logistic Regression

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

# 1. Preprocessing: Convert text to numerical vectors (Bag of Words)
# We limit to the top 5000 most frequent words to keep it simple
vectorizer = CountVectorizer(stop_words='english', max_features=5000)

# Fit on training data, then transform valid and test data
X_train = vectorizer.fit_transform(train_df['text'])
X_valid = vectorizer.transform(valid_df['text'])
X_test = vectorizer.transform(test_df['text'])

y_train = train_df['label']
y_valid = valid_df['label']
y_test = test_df['label']

# 2. Model: Train a simple Logistic Regression model
baseline_model = LogisticRegression(max_iter=1000, random_state=42)
baseline_model.fit(X_train, y_train)

# 3. Evaluation: specific metrics on validation set
valid_preds = baseline_model.predict(X_valid)

print(f"Baseline Validation Accuracy: {accuracy_score(y_valid, valid_preds):.4f}")
print(f"Baseline Validation F1 Score: {f1_score(y_valid, valid_preds):.4f}")

Baseline Validation Accuracy: 0.7570
Baseline Validation F1 Score: 0.7464


Feature Engineering with TF-IDF and N-grams

Idea: weight words to give less importance to common words and more importance to unique words

-> This might signal sarcasm?

N-gram Idea: model seeing pairs of words together might be important for sarcasm b/c gives more context

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. Preprocessing: Use TF-IDF and include Bigrams (1-word and 2-word combinations)
# We increase max_features slightly to accommodate new bigrams
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=10000, ngram_range=(1, 2))

X_train_tfidf = tfidf_vectorizer.fit_transform(train_df['text'])
X_valid_tfidf = tfidf_vectorizer.transform(valid_df['text'])
X_test_tfidf = tfidf_vectorizer.transform(test_df['text'])

# 2. Model: Retrain Logistic Regression on these new features
tfidf_model = LogisticRegression(max_iter=1000, random_state=42)
tfidf_model.fit(X_train_tfidf, y_train)

# 3. Evaluation
valid_preds_tfidf = tfidf_model.predict(X_valid_tfidf)

print(f"TF-IDF + Bigram Accuracy: {accuracy_score(y_valid, valid_preds_tfidf):.4f}")
print(f"TF-IDF + Bigram F1 Score: {f1_score(y_valid, valid_preds_tfidf):.4f}")

TF-IDF + Bigram Accuracy: 0.7737
TF-IDF + Bigram F1 Score: 0.7632


Support Vector Machine
Idea: Good at classification tasks

In [4]:
from sklearn.svm import LinearSVC

# 1. Model: Support Vector Machine (Linear Kernel)
# LinearSVC is faster and often better for text than standard SVC
svm_model = LinearSVC(random_state=42, max_iter=10000)
svm_model.fit(X_train_tfidf, y_train)

# 2. Evaluation
valid_preds_svm = svm_model.predict(X_valid_tfidf)

print(f"SVM Validation Accuracy: {accuracy_score(y_valid, valid_preds_svm):.4f}")
print(f"SVM Validation F1 Score: {f1_score(y_valid, valid_preds_svm):.4f}")

SVM Validation Accuracy: 0.7835
SVM Validation F1 Score: 0.7750


/Users/dylanwong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


LSTM: process text as a sequence rather than Bag of Words

Idea: capture more structure in the sarcasm string

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# 1. Preprocessing: Convert text to sequences of integers
# We limit the vocab to 10,000 words and sequence length to 100
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(train_df['text'])

X_train_seq = pad_sequences(tokenizer.texts_to_sequences(train_df['text']), maxlen=100)
X_valid_seq = pad_sequences(tokenizer.texts_to_sequences(valid_df['text']), maxlen=100)
X_test_seq = pad_sequences(tokenizer.texts_to_sequences(test_df['text']), maxlen=100)

# 2. Model: Define a simple LSTM network 
model = Sequential([
    Embedding(input_dim=10000, output_dim=32, input_length=100),
    LSTM(32),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 3. Training
model.fit(X_train_seq, y_train, epochs=5, batch_size=64, validation_data=(X_valid_seq, y_valid))

# 4. Evaluation
lstm_probs = model.predict(X_valid_seq)
lstm_preds = (lstm_probs > 0.5).astype(int)

print(f"LSTM Validation Accuracy: {accuracy_score(y_valid, lstm_preds):.4f}")
print(f"LSTM Validation F1 Score: {f1_score(y_valid, lstm_preds):.4f}")

Epoch 1/5


/Users/dylanwong/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


336/336 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - accuracy: 0.7770 - loss: 0.4632 - val_accuracy: 0.8506 - val_loss: 0.3589
Epoch 2/5
336/336 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.9004 - loss: 0.2505 - val_accuracy: 0.8408 - val_loss: 0.3695
Epoch 3/5
336/336 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.9353 - loss: 0.1762 - val_accuracy: 0.8268 - val_loss: 0.4283
Epoch 4/5
336/336 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.9530 - loss: 0.1342 - val_accuracy: 0.8324 - val_loss: 0.4663
Epoch 5/5
336/336 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.9636 - loss: 0.1059 - val_accuracy: 0.8310 - val_loss: 0.5059
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
LSTM Validation Accuracy: 0.8310
LSTM Validation F1 Score: 0.8322


BiLSTM to read strings bidirectionally

Idea: typically better to gain more information.

Optimization: Dropout so that the network is not overly reliant on specific features (prevents overfitting)

In [6]:
from tensorflow.keras.layers import Bidirectional, Dropout

# 1. Model: Define a Bidirectional LSTM with Dropout
# We wrap the LSTM layer in 'Bidirectional'
bilstm_model = Sequential([
    Embedding(input_dim=10000, output_dim=32, input_length=100),
    Bidirectional(LSTM(32)),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

bilstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 2. Training
# We use the same sequence data (X_train_seq) prepared in the previous step
bilstm_model.fit(X_train_seq, y_train, epochs=5, batch_size=64, validation_data=(X_valid_seq, y_valid))

# 3. Evaluation
bilstm_probs = bilstm_model.predict(X_valid_seq)
bilstm_preds = (bilstm_probs > 0.5).astype(int)

print(f"Bi-LSTM Validation Accuracy: {accuracy_score(y_valid, bilstm_preds):.4f}")
print(f"Bi-LSTM Validation F1 Score: {f1_score(y_valid, bilstm_preds):.4f}")

Epoch 1/5


/Users/dylanwong/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


336/336 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step - accuracy: 0.7620 - loss: 0.4814 - val_accuracy: 0.8492 - val_loss: 0.3483
Epoch 2/5
336/336 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - accuracy: 0.9009 - loss: 0.2549 - val_accuracy: 0.8450 - val_loss: 0.3517
Epoch 3/5
336/336 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - accuracy: 0.9357 - loss: 0.1814 - val_accuracy: 0.8478 - val_loss: 0.3977
Epoch 4/5
336/336 ━━━━━━━━━━━━━━━━━━━━ 11s 31ms/step - accuracy: 0.9519 - loss: 0.1392 - val_accuracy: 0.8310 - val_loss: 0.4543
Epoch 5/5
336/336 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - accuracy: 0.9646 - loss: 0.1045 - val_accuracy: 0.8352 - val_loss: 0.5630
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Bi-LSTM Validation Accuracy: 0.8352
Bi-LSTM Validation F1 Score: 0.8309


Why did the accuracy go down bro :(

Build Ensemble Model

In [7]:
# 1. Gather predictions from our top 3 models
# Note: We flatten the neural network arrays to make them match the SVM's shape
pred_1 = valid_preds_svm
pred_2 = lstm_preds.flatten()
pred_3 = bilstm_preds.flatten()

# 2. Voting Logic: Sum the predictions
# If sum is 2 or 3, it means the majority voted 1 (Sarcasm)
total_votes = pred_1 + pred_2 + pred_3
ensemble_preds = (total_votes >= 2).astype(int)

# 3. Evaluation
print(f"Ensemble Validation Accuracy: {accuracy_score(y_valid, ensemble_preds):.4f}")
print(f"Ensemble Validation F1 Score: {f1_score(y_valid, ensemble_preds):.4f}")

Ensemble Validation Accuracy: 0.8352
Ensemble Validation F1 Score: 0.8324


Evaluation on Test Set

In [8]:
from sklearn.metrics import classification_report, confusion_matrix

# 1. Get predictions for the Test Set from all 3 models
# SVM (uses TF-IDF features)
pred_test_svm = svm_model.predict(X_test_tfidf)

# LSTM (uses Sequence features)
pred_test_lstm = (model.predict(X_test_seq) > 0.5).astype(int).flatten()

# Bi-LSTM (uses Sequence features)
pred_test_bilstm = (bilstm_model.predict(X_test_seq) > 0.5).astype(int).flatten()

# 2. Ensemble Voting (Majority Vote)
# Sum the predictions (0 or 1). If sum is 2 or 3, majority is 1.
test_votes = pred_test_svm + pred_test_lstm + pred_test_bilstm
pred_test_ensemble = (test_votes >= 2).astype(int)

# 3. Report detailed metrics
print("Final Evaluation on Test Set:")
print(classification_report(test_df['label'], pred_test_ensemble, digits=4))

# 4. Confusion Matrix (Row: True, Col: Predicted)
print("\nConfusion Matrix:")
print(confusion_matrix(test_df['label'], pred_test_ensemble))

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Final Evaluation on Test Set:
              precision    recall  f1-score   support

           0     0.8779    0.8612    0.8695       526
           1     0.8378    0.8568    0.8472       440

    accuracy                         0.8592       966
   macro avg     0.8578    0.8590    0.8583       966
weighted avg     0.8596    0.8592    0.8593       966


Confusion Matrix:
[[453  73]
 [ 63 377]]


Save Weights

# Trying Improvements to the model below.

Text Preprocessing: 

In [9]:
import re
from scipy.sparse import hstack

# 1. Feature Extraction (Run this on RAW text to capture capitalization/punctuation)
def extract_features(df):
    features = pd.DataFrame()
    # Punctuation counts (sarcasm indicators)
    features['exclamation_count'] = df['text'].str.count('!')
    features['question_count'] = df['text'].str.count('\?')
    features['ellipsis_count'] = df['text'].str.count(r'\.\.\.')
    
    # Capitalization (sarcasm often uses ALL CAPS or Mixed Caps)
    features['capital_ratio'] = df['text'].apply(lambda x: sum(1 for c in x if c.isupper()) / (len(x) + 1))
    features['has_all_caps_word'] = df['text'].str.contains(r'\b[A-Z]{2,}\b').astype(int)
    
    # Length metrics
    features['text_length'] = df['text'].str.len()
    features['word_count'] = df['text'].str.split().str.len()
    
    return features

# Extract features now
train_features = extract_features(train_df)
valid_features = extract_features(valid_df)
test_features = extract_features(test_df)

In [10]:
# 2. Text Preprocessing
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+', '', text) # Remove URLs
    text = re.sub(r'\s+', ' ', text).strip()    # Clean whitespace
    return text

# Apply preprocessing (This modifies the data for subsequent steps)
train_df['clean_text'] = train_df['text'].apply(preprocess_text)
valid_df['clean_text'] = valid_df['text'].apply(preprocess_text)
test_df['clean_text'] = test_df['text'].apply(preprocess_text)

TF-IDF on pre-processed data

In [11]:
# 3. New TF-IDF on Cleaned Text
# Re-running vectorizer on the cleaner text
tfidf_clean = TfidfVectorizer(stop_words='english', max_features=10000, ngram_range=(1, 2))
X_train_tfidf_clean = tfidf_clean.fit_transform(train_df['clean_text'])
X_valid_tfidf_clean = tfidf_clean.transform(valid_df['clean_text'])
X_test_tfidf_clean = tfidf_clean.transform(test_df['clean_text'])

# 4. Combine Features (TF-IDF + Manual Features)
X_train_combined = hstack([X_train_tfidf_clean, train_features])
X_valid_combined = hstack([X_valid_tfidf_clean, valid_features])
X_test_combined = hstack([X_test_tfidf_clean, test_features])

# 5. Train Improved SVM
svm_model_improved = LinearSVC(random_state=42, max_iter=10000, C=1.0)
svm_model_improved.fit(X_train_combined, y_train)

# Evaluation
valid_preds_svm_improved = svm_model_improved.predict(X_valid_combined)
print(f"Improved SVM Accuracy: {accuracy_score(y_valid, valid_preds_svm_improved):.4f}")
print(f"Improved SVM F1 Score: {f1_score(y_valid, valid_preds_svm_improved):.4f}")

/Users/dylanwong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Improved SVM Accuracy: 0.7751
Improved SVM F1 Score: 0.7504


/Users/dylanwong/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Stacked BiDirectional LSTM + Early Stopping

In [12]:
from tensorflow.keras.callbacks import EarlyStopping

# 1. Update Sequences: Use the 'clean_text' from the previous step
# We re-fit the tokenizer on the cleaner text
tokenizer_clean = Tokenizer(num_words=10000)
tokenizer_clean.fit_on_texts(train_df['clean_text'])

X_train_seq_clean = pad_sequences(tokenizer_clean.texts_to_sequences(train_df['clean_text']), maxlen=100)
X_valid_seq_clean = pad_sequences(tokenizer_clean.texts_to_sequences(valid_df['clean_text']), maxlen=100)
X_test_seq_clean = pad_sequences(tokenizer_clean.texts_to_sequences(test_df['clean_text']), maxlen=100)

# 2. Improved Model: Stacked Bi-LSTM
# We stack two Bidirectional LSTM layers to learn more complex patterns
bilstm_improved = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=100),
    Bidirectional(LSTM(64, return_sequences=True)), # Return sequences is required to stack another LSTM
    Dropout(0.3),
    Bidirectional(LSTM(32)),
    Dropout(0.5),
    Dense(16, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

bilstm_improved.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 3. Training with Early Stopping
# Stop training if validation loss doesn't improve for 3 epochs
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

bilstm_improved.fit(
    X_train_seq_clean, y_train,
    epochs=10,
    batch_size=64,
    validation_data=(X_valid_seq_clean, y_valid),
    callbacks=[early_stop]
)

# 4. Evaluation
probs_bilstm_improved = bilstm_improved.predict(X_valid_seq_clean)
preds_bilstm_improved = (probs_bilstm_improved > 0.5).astype(int)

print(f"Improved Bi-LSTM Accuracy: {accuracy_score(y_valid, preds_bilstm_improved):.4f}")
print(f"Improved Bi-LSTM F1 Score: {f1_score(y_valid, preds_bilstm_improved):.4f}")

Epoch 1/10


/Users/dylanwong/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


336/336 ━━━━━━━━━━━━━━━━━━━━ 45s 124ms/step - accuracy: 0.7885 - loss: 0.4463 - val_accuracy: 0.8589 - val_loss: 0.3360
Epoch 2/10
336/336 ━━━━━━━━━━━━━━━━━━━━ 48s 141ms/step - accuracy: 0.9146 - loss: 0.2319 - val_accuracy: 0.8492 - val_loss: 0.3866
Epoch 3/10
336/336 ━━━━━━━━━━━━━━━━━━━━ 47s 139ms/step - accuracy: 0.9541 - loss: 0.1324 - val_accuracy: 0.8394 - val_loss: 0.4783
Epoch 4/10
336/336 ━━━━━━━━━━━━━━━━━━━━ 49s 147ms/step - accuracy: 0.9751 - loss: 0.0783 - val_accuracy: 0.8142 - val_loss: 0.7130
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step
Improved Bi-LSTM Accuracy: 0.8589
Improved Bi-LSTM F1 Score: 0.8571


CNN for text classification to spot N-gram patterns

Idea: detect phrases for sarcasm

In [13]:
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D

# 1. Model: 1D Convolutional Neural Network
# - Conv1D with kernel_size=5 looks at 5-word windows to find sarcastic phrases
# - GlobalMaxPooling1D keeps only the strongest signal found in the text
cnn_model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=100),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(16, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 2. Training
# We use the same 'clean' sequences and early stopping as before
cnn_model.fit(
    X_train_seq_clean, y_train,
    epochs=10,
    batch_size=64,
    validation_data=(X_valid_seq_clean, y_valid),
    callbacks=[early_stop]
)

# 3. Evaluation
probs_cnn = cnn_model.predict(X_valid_seq_clean)
preds_cnn = (probs_cnn > 0.5).astype(int)

print(f"CNN Validation Accuracy: {accuracy_score(y_valid, preds_cnn):.4f}")
print(f"CNN Validation F1 Score: {f1_score(y_valid, preds_cnn):.4f}")

Epoch 1/10


/Users/dylanwong/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


336/336 ━━━━━━━━━━━━━━━━━━━━ 7s 17ms/step - accuracy: 0.8003 - loss: 0.4306 - val_accuracy: 0.8547 - val_loss: 0.3322
Epoch 2/10
336/336 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.9223 - loss: 0.2080 - val_accuracy: 0.8506 - val_loss: 0.3709
Epoch 3/10
336/336 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.9741 - loss: 0.0840 - val_accuracy: 0.8338 - val_loss: 0.4929
Epoch 4/10
336/336 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.9929 - loss: 0.0279 - val_accuracy: 0.8324 - val_loss: 0.6885
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
CNN Validation Accuracy: 0.8547
CNN Validation F1 Score: 0.8567


Improved Model Evaluation on Test Set

In [14]:
from sklearn.metrics import classification_report, confusion_matrix

# --- 1. Validation Set Ensemble ---

# Get predictions from the 3 improved models on Validation Data
# SVM (uses Combined Sparse Features)
val_pred_svm = svm_model_improved.predict(X_valid_combined)

# Bi-LSTM (uses Clean Sequences)
val_probs_bilstm = bilstm_improved.predict(X_valid_seq_clean)
val_pred_bilstm = (val_probs_bilstm > 0.5).astype(int).flatten()

# CNN (uses Clean Sequences)
val_probs_cnn = cnn_model.predict(X_valid_seq_clean)
val_pred_cnn = (val_probs_cnn > 0.5).astype(int).flatten()

# Majority Vote Ensemble
val_votes = val_pred_svm + val_pred_bilstm + val_pred_cnn
val_pred_ensemble = (val_votes >= 2).astype(int)

print(f"Final Ensemble Validation Accuracy: {accuracy_score(y_valid, val_pred_ensemble):.4f}")
print(f"Final Ensemble Validation F1 Score: {f1_score(y_valid, val_pred_ensemble):.4f}")

# --- 2. Test Set Evaluation (The Deliverable) ---

# Get predictions on Test Data
test_pred_svm = svm_model_improved.predict(X_test_combined)

test_probs_bilstm = bilstm_improved.predict(X_test_seq_clean)
test_pred_bilstm = (test_probs_bilstm > 0.5).astype(int).flatten()

test_probs_cnn = cnn_model.predict(X_test_seq_clean)
test_pred_cnn = (test_probs_cnn > 0.5).astype(int).flatten()

# Majority Vote Ensemble
test_votes = test_pred_svm + test_pred_bilstm + test_pred_cnn
test_pred_ensemble = (test_votes >= 2).astype(int)

# Report
print("FINAL TEST SET RESULTS:")
print(classification_report(test_df['label'], test_pred_ensemble, digits=4))

print("\nConfusion Matrix:")
print(confusion_matrix(test_df['label'], test_pred_ensemble))

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Final Ensemble Validation Accuracy: 0.8589
Final Ensemble Validation F1 Score: 0.8571
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
FINAL TEST SET RESULTS:
              precision    recall  f1-score   support

           0     0.8689    0.8821    0.8755       526
           1     0.8565    0.8409    0.8486       440

    accuracy                         0.8634       966
   macro avg     0.8627    0.8615    0.8620       966
weighted avg     0.8633    0.8634    0.8632       966


Confusion Matrix:
[[464  62]
 [ 70 370]]


In [15]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

# --- 1. Get Predictions for the Test Set ---

# Model A: ORIGINAL SVM (Using TF-IDF features from snippet 1)
# Assuming 'svm_model' is your original model
test_pred_svm = svm_model.predict(X_test_tfidf)

# Model B: BI-LSTM (Using clean sequences from snippet 2)
test_probs_bilstm = bilstm_improved.predict(X_test_seq_clean)
test_pred_bilstm = (test_probs_bilstm > 0.5).astype(int).flatten()

# Model C: CNN (Using clean sequences from snippet 2)
test_probs_cnn = cnn_model.predict(X_test_seq_clean)
test_pred_cnn = (test_probs_cnn > 0.5).astype(int).flatten()

# --- 2. Voting Logic (Hard Voting) ---

# Sum the binary predictions (0 or 1)
# Possible sums: 0, 1 (Majority 0) | 2, 3 (Majority 1)
test_votes = test_pred_svm + test_pred_bilstm + test_pred_cnn
final_ensemble_preds = (test_votes >= 2).astype(int)

# --- 3. Final Evaluation ---

print("=== ENSEMBLE RESULTS (Original SVM + Bi-LSTM + CNN) ===")
print(f"Accuracy: {accuracy_score(test_df['label'], final_ensemble_preds):.4f}")
print(f"F1 Score: {f1_score(test_df['label'], final_ensemble_preds):.4f}")

print("\nDetailed Classification Report:")
print(classification_report(test_df['label'], final_ensemble_preds, digits=4))

print("\nConfusion Matrix:")
print(confusion_matrix(test_df['label'], final_ensemble_preds))

31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
=== ENSEMBLE RESULTS (Original SVM + Bi-LSTM + CNN) ===
Accuracy: 0.8654
F1 Score: 0.8526

Detailed Classification Report:
              precision    recall  f1-score   support

           0     0.8779    0.8745    0.8762       526
           1     0.8507    0.8545    0.8526       440

    accuracy                         0.8654       966
   macro avg     0.8643    0.8645    0.8644       966
weighted avg     0.8655    0.8654    0.8654       966


Confusion Matrix:
[[460  66]
 [ 64 376]]
